In [2]:
import requests
import pandas as pd
import os

df_esempio = pd.read_csv('diabetic_data_filtered_250.csv')
esempio_csv = df_esempio.head(30).to_string()
colonne_originali = ", ".join(df_esempio.columns)

print(colonne_originali)

encounter_id, patient_nbr, race, gender, age, weight, admission_type_id, discharge_disposition_id, admission_source_id, time_in_hospital, payer_code, medical_specialty, num_lab_procedures, num_procedures, num_medications, number_outpatient, number_emergency, number_inpatient, diag_1, diag_2, diag_3, number_diagnoses, max_glu_serum, A1Cresult, metformin, repaglinide, nateglinide, chlorpropamide, glimepiride, acetohexamide, glipizide, glyburide, tolbutamide, pioglitazone, rosiglitazone, acarbose, miglitol, troglitazone, tolazamide, examide, citoglipton, insulin, glyburide-metformin, glipizide-metformin, glimepiride-pioglitazone, metformin-rosiglitazone, metformin-pioglitazone, change, diabetesMed, readmitted


# Requests OLLAMA

In [10]:
from io import StringIO

OLLAMA_ENDPOINT = "http://localhost:11434/api/generate"

prompt = f"""
GENERA UN SOLO RECORD CSV per il seguente paziente diabetico:

DATI DEL PAZIENTE:
- Nome: Emanuele Ragozzini
- Età: 25 anni
- Sesso: Maschio
- Ricoverato il: 12 maggio 2025 al Moscati di Avellino
- Valore di glucosio iniziale: superiore a 300 mg/dL
- Storia clinica: Ricevuto medicinale errato, peggioramento, iperglicemia, ricovero, valore glucosio più alto, nuova diagnosi, medicinali corretti, diagnosi finale di diabete

REQUISITI TECNICI:
1. USA ESATTAMENTE queste colonne nel CSV (nessuna di più, nessuna di meno):
{colonne_originali}

2. FORMATO DI ESEMPIO (ignora i valori, concentrati sul formato):
{df_esempio.head(1).to_csv(index=False)}

3. REGOLE RIGIDE:
- GENERA UN SOLO RECORD (una sola riga dati oltre all'intestazione)
- INCLUDI TUTTE LE COLONNE specificate, nell'ordine esatto
- OGNI CELLA DEVE AVERE UN VALORE (niente null o vuoto)
- USA LA VIRGOLA (,) come separatore
- USA LE VIRGOLETTE per valori con spazi o virgole

RISPONDI ESCLUSIVAMENTE CON IL CSV VALIDO:
- Prima riga: intestazione con i nomi delle colonne
- Seconda riga: i dati del paziente Emanuele Ragozzini
- NESSUN altro testo o spiegazione

ATTENZIONE: Il formato deve essere perfettamente compatibile con pandas.read_csv()
"""



modello = "llama3.2:latest"
payload = {
    "model": modello,
    "prompt": prompt,
    "stream": False
}

response = requests.post(OLLAMA_ENDPOINT, json=payload)

if response.status_code == 200:
    result = response.json()['response']
    print("Risposta grezza ricevuta:")
    print(result)

    # Pulisci la risposta per estrarre solo le righe CSV valide
    csv_lines = []
    for line in result.split('\n'):
        line = line.strip()
        # Ignora eventuali delimitatori di blocco di codice
        if line.startswith('```') and ('csv' in line or not csv_lines):
            continue
        if line == '```':
            continue
        # Ignora linee di commento o vuote
        if not line or line.startswith('#') or line.startswith('*'):
            continue
        # Aggiungi la riga
        csv_lines.append(line)

    csv_content = '\n'.join(csv_lines)

    print("\nContenuto CSV estratto:")
    print(csv_content)

    try:
        # Converti in DataFrame con gestione esplicita dei valori nulli
        df = pd.read_csv(StringIO(csv_content), na_values=['null', 'NULL', 'None', 'none', ''])

        print("\nColonne trovate:", df.columns.tolist())
        print("Colonne attese:", df_esempio.columns.tolist())

        # Verifica che tutte le colonne richieste siano presenti
        if not all(col in df.columns for col in df_esempio.columns):
            print("ERRORE: Colonne mancanti nella risposta")
            missing_cols = set(df_esempio.columns) - set(df.columns)
            print(f"Colonne mancanti: {missing_cols}")
        else:
            # Riordina le colonne per essere sicuri
            df = df[df_esempio.columns]

            # Verifica che non ci siano valori null
            null_count = df.isnull().sum().sum()
            if null_count > 0:
                print(f"ATTENZIONE: Trovati {null_count} valori nulli nel CSV")
                print(df.isnull().sum())

            # Salva il file
            output_path = 'StoryTellingPrompt/llama3.2_storytelling_synthetic.csv'
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            df.to_csv(output_path, index=False)
            print(f"File CSV salvato correttamente in: {output_path}")
    except Exception as e:
        print(f"Errore nel processing della risposta: {e}")
        print("Risposta CSV problematica:")
        print(csv_content)


Risposta grezza ricevuta:
```
encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
2278392,8222157,Maschio,25,"Superiore a 300 mg/dL",No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,No,Yes,Yes,No,Yes,Yes,Yes,No,Yes,No,Yes,No,No,Yes,No,Yes,No,Yes,No,No,No,No,Yes,No,No,No,Yes,
```

Contenuto CSV estratto:
encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_dis

# Requests GROQ

In [10]:
from io import StringIO
import os
import requests
import pandas as pd
import dotenv

dotenv.load_dotenv()
GROQ_API_KEY = os.getenv('MISTRAL_API_KEY')
GROQ_ENDPOINT = "https://api.groq.com/openai/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json"
}

prompt = f"""Sei un generatore di dati sintetici per un ospedale. I dati riguardano pazienti diabetici.

QUESTO È L'ESEMPIO ESATTO DEL FORMATO DA SEGUIRE:

{esempio_csv}

REQUISITI FONDAMENTALI:
1. Genera ESCLUSIVAMENTE un file CSV con 20 nuove righe
2. Il CSV DEVE contenere ESATTAMENTE queste colonne (TUTTE OBBLIGATORIE), nello stesso ordine:
{colonne_originali}
3. TUTTE le celle devono avere un valore (OBBLIGATORIAMENTE), NON POSSONO ESSERCI CELLE VUOTE

3. REGOLE CRITICHE:
- NON aggiungere colonne extra
- NON omettere nessuna colonna
- NON aggiungere testo o commenti
- NON utilizzare punti e virgola (;) come separatore, usa la virgola (,)
- TUTTE le celle devono avere un valore, non possono essere vuote o null
- Mantieni gli stessi tipi di dati per ogni colonna come nell'esempio
- Mantieni gli stessi formati dei valori come nell'esempio

RISPONDI ESCLUSIVAMENTE CON LE RIGHE CSV, NIENT'ALTRO
ATTENZIONE: Se anche una sola cella è vuota o contiene '<null>', il file sarà SCARTATO. Genera dati COMPLETI, coerenti, senza valori nulli.
"""

payload = {
    "model": "llama-3.1-8b-instant",
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ],
    "temperature": 0.4,
    "max_tokens": 12000
}

# Effettua la richiesta
response = requests.post(GROQ_ENDPOINT, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()['choices'][0]['message']['content']
    print(result)

    csv_lines = [line for line in result.split('\n') if line.strip() and not line.startswith('*') and not line.startswith('#')]

    csv_content = '\n'.join(csv_lines)

    try:
        # Converti in DataFrame
        df = pd.read_csv(StringIO(csv_content))

        if not all(col in df.columns for col in df_esempio.columns):
            print("ERRORE: Colonne mancanti nella risposta")
            missing_cols = set(df_esempio.columns) - set(df.columns)
            print(f"Colonne mancanti: {missing_cols}")
        else:
            df = df[df_esempio.columns]

            # Salva il file
            output_path = 'StoryTellingPrompt/llama-3.1-8b-instant_synthetic_diabetes_data.csv'
            os.makedirs('StoryTellingPrompt', exist_ok=True)
            df.to_csv(output_path, index=False)
            print(f"File CSV salvato correttamente in: {output_path}")

    except Exception as e:
        print(f"Errore nel processing della risposta: {e}")
        print("Risposta ricevuta:")
        print(result)
else:
    print(f"Errore nella richiesta: {response.status_code}")
    print(response.text)

1,  323911,  55629189, Caucasian, Female, [10-20),  250.01,  1,  1,  7,  3,  Pediatrics-Endocrinology,  41,  0,  1,  0,  0,  0,  250.83,  7.5,  7.3,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  No,  

# Request MISTRAL

In [35]:
from io import StringIO
import os
import requests
import pandas as pd
import dotenv

dotenv.load_dotenv()
MISTRAL_API_KEY = os.getenv('MISTRAL_API_KEY')
MISTRAL_ENDPOINT = "https://api.mistral.ai/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {MISTRAL_API_KEY}",
    "Content-Type": "application/json"
}

prompt = f"""Sei un generatore di dati sintetici per un ospedale. I dati riguardano pazienti diabetici.

QUESTO È L'ESEMPIO ESATTO DEL FORMATO DA SEGUIRE:

{esempio_csv}

REQUISITI FONDAMENTALI:
1. Genera ESCLUSIVAMENTE un file CSV con 20 nuove righe
2. Il CSV DEVE contenere ESATTAMENTE queste colonne (TUTTE OBBLIGATORIE), nello stesso ordine:
{colonne_originali}
3. TUTTE le celle devono avere un valore (OBBLIGATORIAMENTE), NON POSSONO ESSERCI CELLE VUOTE

3. REGOLE CRITICHE:
- NON aggiungere colonne extra
- NON omettere nessuna colonna
- NON aggiungere testo o commenti
- NON utilizzare punti e virgola (;) come separatore, usa la virgola (,)
- TUTTE le celle devono avere un valore, non possono essere vuote o null
- Mantieni gli stessi tipi di dati per ogni colonna come nell'esempio
- Mantieni gli stessi formati dei valori come nell'esempio

RISPONDI ESCLUSIVAMENTE CON LE RIGHE CSV, NIENT'ALTRO
"""

payload = {
    "model": "mistral-small-2503",
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ],
    "temperature": 0.1,
    "max_tokens": 12000,
}

response = requests.post(MISTRAL_ENDPOINT, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()['choices'][0]['message']['content']
    print(result)
    result = result.replace('\_', '_')

    csv_lines = [line for line in result.split('\n') if line.strip() and not line.startswith('*') and not line.startswith('#')]

    csv_content = '\n'.join(csv_lines)

    try:
        # Converti in DataFrame
        df = pd.read_csv(StringIO(csv_content))

        if not all(col in df.columns for col in df_esempio.columns):
            print("ERRORE: Colonne mancanti nella risposta")
            missing_cols = set(df_esempio.columns) - set(df.columns)
            print(f"Colonne mancanti: {missing_cols}")
        else:
            df = df[df_esempio.columns]

            # Salva il file
            output_path = 'StoryTellingPrompt/mistral-small-2503_synthetic_diabetes_data.csv'
            os.makedirs('StoryTellingPrompt', exist_ok=True)
            df.to_csv(output_path, index=False)
            print(f"File CSV salvato correttamente in: {output_path}")

    except Exception as e:
        print(f"Errore nel processing della risposta: {e}")
        print("Risposta ricevuta:")
        print(result)
else:
    print(f"Errore nella richiesta: {response.status_code}")
    print(response.text)

```csv
encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
123456,12345678,Caucasian,Female,[30-40),?,1,25,7,?,?,Pediatrics-Endocrinology,35,0,12,0,0,0,250.5,255,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,N